In [17]:
# load the libraries

import urllib2, urllib
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import urllib2
import json

In [18]:
df = pd.read_csv('test.txt',header=None) # Here we are reading the 10K submission links

links = list(df[0])[0:5]
links

['https://www.sec.gov/Archives/edgar/data/1060822/000106082217000008/a10k-crix20161231.htm',
 'https://www.sec.gov/Archives/edgar/data/1050797/000105079717000013/colm-20161231x10k.htm',
 'https://www.sec.gov/Archives/edgar/data/1334036/000162828017002068/crox-20161231x10k.htm',
 'https://www.sec.gov/Archives/edgar/data/723603/000115752317001944/a51582973.htm',
 'https://www.sec.gov/Archives/edgar/data/910521/000091052117000015/a10ka.htm']

In [19]:
# get the list of item 1A: risk factors

risk1 = []

for i in range(len(links)):
    html = urllib2.urlopen(links[i]).read()
    index = re.search(r'\b(1A)\b', html) # searches the submission file for the exact mention of the term '1A'
    html = html[index.start()+3:] #cuts the html after the first 1A occurence, which is after the table of contents
    index = re.search(r'\b(1A)\b', html) # finds 1A in the html after the table of contents
    html = html[index.start():] # cuts off the previous html part
    index = re.search(r'\b(1B)\b', html) # finds the exact occurence of 1B
    risk1.append(html[:index.start()]) # stores the start of 1A to start of 1B in a string

risk2 = []

for i in range(len(risk1)):
    soup = BeautifulSoup(risk1[i])

    # get text
    text = soup.get_text() # getting all the text from the soup

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    #whenever there is an empty string, the following if condition would render false
    text = '\n'.join(chunk for chunk in chunks if chunk)

    risk2.append(text.encode('utf-8'))

AttributeError: 'NoneType' object has no attribute 'start'

In [43]:
# get the list of item 7: Management Discussion

md1 = []

for i in range(len(links)):
    html = urllib2.urlopen(links[i]).read()
    index = re.search('Item 7.', html) # searches the submission file for the exact mention of the term 'Item 7.'
    html = html[index.start()+3:] #cuts the html after the first Item 7. occurence, which is after the table of contents
    index = re.search('ITEM 7.', html) # finds ITEM 7. in the html after the table of contents
    html = html[index.start():] # cuts off the previous html part
    index = re.search('ITEM 7A.', html) # finds the exact occurence of ITEM 7A.
    md1.append(html[:index.start()]) # stores the start of 1A to start of 1B in a string

md2 = []

for i in range(len(md1)):
    soup = BeautifulSoup(md1[i])

    # get text
    text = soup.get_text() # getting all the text from the soup

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    #whenever there is an empty string, the following if condition would render false
    text = '\n'.join(chunk for chunk in chunks if chunk)

    md2.append(text.encode('utf-8'))

md2

['ITEM 7.MANAGEMENT\'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONSOverviewWe are a diversified chemical company serving agricultural, consumer and industrial markets globally with innovative solutions, applications and market-leading products. We operate in three distinct business segments: FMC Agricultural Solutions, FMC Health and Nutrition and FMC Lithium. Our FMC Agricultural Solutions segment develops, markets and sells all three major classes of crop protection chemicals: insecticides, herbicides and fungicides. These products are used in agriculture to enhance crop yield and quality by controlling a broad spectrum of insects, weeds and disease, as well as in non-agricultural markets for pest control. The FMC Health and Nutrition segment focuses on nutritional ingredients, health excipients, and functional health ingredients. Nutritional ingredients are used to enhance texture, color, structure and physical stability. Health excipients are used for bi

In [6]:
# get the list of item 7A: Market Risk

market1 = []

for i in range(len(links)):
    html = urllib2.urlopen(links[i]).read()
    index = re.search('Item 7A.', html) # searches the submission file for the exact mention of the term 'Item 7.'
    html = html[index.start()+3:] #cuts the html after the first Item 7. occurence, which is after the table of contents
    index = re.search('ITEM 7A.', html) # finds ITEM 7. in the html after the table of contents
    html = html[index.start():] # cuts off the previous html part
    index = re.search('ITEM 8.', html) # finds the exact occurence of ITEM 7A.
    market1.append(html[:index.start()]) # stores the start of 1A to start of 1B in a string

market2 = []

for i in range(len(market1)):
    soup = BeautifulSoup(market1[i])

    # get text
    text = soup.get_text() # getting all the text from the soup

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    #whenever there is an empty string, the following if condition would render false
    text = '\n'.join(chunk for chunk in chunks if chunk)

    market2.append(text.encode('utf-8'))

In [34]:
# convert list of risk factor into list of score

for i in range(len(risk2)):
    
    sentiment1 = []
    keyphrases1 = []
    total_keys1 = []
    avg_score1 = []
    
    final1 = []
    text = risk2[i]
    for x in range(0, len(text), 2000):
        if len(text)-x > 2000:
            parts = text[x:x+2000]
        else:
            parts = text[x:]

        #Here we have broken down our text in to 2000 character chunks

        # Azure portal URL.
        base_url = 'https://westus.api.cognitive.microsoft.com/'
        # The unique account key goes here.
        account_key = '01f5b19c9bd9425ab5c17dfaf2a497ed'

        headers = {'Content-Type':'application/json', 'Ocp-Apim-Subscription-Key':account_key}

        input_texts = '{"documents":[{"id":"1","text":"'+parts+'"}]}' 
        #Here, instead of manually inputting any codes, we have put in the 2000 character chunk

        # Detect key phrases.
        batch_keyphrase_url = base_url + 'text/analytics/v2.0/keyPhrases'
        req = urllib2.Request(batch_keyphrase_url, input_texts, headers) 
        response = urllib2.urlopen(req)
        result = response.read()
        keyph = json.loads(result)

        for phrases in keyph['documents']:
            keyphrases1.append(phrases['keyPhrases'])

        # Detect sentiment.
        batch_sentiment_url = base_url + 'text/analytics/v2.0/sentiment'
        req = urllib2.Request(batch_sentiment_url, input_texts, headers) 
        response = urllib2.urlopen(req)
        result = response.read()
        senti = json.loads(result)

        for sentiment_analysis in senti['documents']:
            sentiment1.append(sentiment_analysis['score']) # The sentiment scores are appended to the sentiment list

    # The keyphrases list will be a list within a list, containing a set of keywords obtained after every single run of 2000
    # characters. So now, we will iterate over these inner lists to get each of the keywords into a final list

    for items in keyphrases1:
        for x in items:
            final1.append(x)

    unique_keys = list(set(final1)) #Using set functions, we clean the list to remove duplicates

    total_keys1.append(unique_keys) #In this list we will be appending the list of unique keywords for each of the 10K submissions

    avg_senti = np.mean(sentiment1) #Here we are calculate the average sentiment score for each of the chunks

    avg_score1.append(avg_senti) # Storing the average sentiment score in the average score list

In [35]:
avg_score1

[0.43455511060628021]

In [42]:
# convert list of Management Discussion into list of score

for i in range(len(md2)):
    
    sentiment2 = []
    keyphrases2 = []
    total_keys2 = []
    avg_score2 = []
    
    final2 = []
    text = md2[i]
    for x in range(0, len(text), 2000):
        if len(text)-x > 2000:
            parts = text[x:x+2000]
        else:
            parts = text[x:]

        #Here we have broken down our text in to 2000 character chunks

        # Azure portal URL.
        base_url = 'https://westus.api.cognitive.microsoft.com/'
        # The unique account key goes here.
        account_key = '01f5b19c9bd9425ab5c17dfaf2a497ed'

        headers = {'Content-Type':'application/json', 'Ocp-Apim-Subscription-Key':account_key}

        input_texts = '{"documents":[{"id":"1","text":"'+parts+'"}]}' 
        #Here, instead of manually inputting any codes, we have put in the 2000 character chunk

        # Detect key phrases.
        batch_keyphrase_url = base_url + 'text/analytics/v2.0/keyPhrases'
        req = urllib2.Request(batch_keyphrase_url, input_texts, headers) 
        response = urllib2.urlopen(req)
        result = response.read()
        keyph = json.loads(result)

        for phrases in keyph['documents']:
            keyphrases2.append(phrases['keyPhrases'])

        # Detect sentiment.
        batch_sentiment_url = base_url + 'text/analytics/v2.0/sentiment'
        req = urllib2.Request(batch_sentiment_url, input_texts, headers) 
        response = urllib2.urlopen(req)
        result = response.read()
        senti = json.loads(result)

        for sentiment_analysis in senti['documents']:
            sentiment2.append(sentiment_analysis['score']) # The sentiment scores are appended to the sentiment list

    # The keyphrases list will be a list within a list, containing a set of keywords obtained after every single run of 2000
    # characters. So now, we will iterate over these inner lists to get each of the keywords into a final list

    for items in keyphrases2:
        for x in items:
            final2.append(x)

    unique_keys = list(set(final2)) #Using set functions, we clean the list to remove duplicates

    total_keys2.append(unique_keys) #In this list we will be appending the list of unique keywords for each of the 10K submissions

    avg_senti = np.mean(sentiment2) #Here we are calculate the average sentiment score for each of the chunks

    avg_score2.append(avg_senti) # Storing the average sentiment score in the average score list

HTTPError: HTTP Error 400: Bad Request

In [36]:
# convert list of Management Discussion into list of score

for i in range(len(market2)):
    
    sentiment3 = []
    keyphrases3 = []
    total_keys3 = []
    avg_score3 = []
    
    final3 = []
    text = market2[i]
    for x in range(0, len(text), 2000):
        if len(text)-x > 2000:
            parts = text[x:x+2000]
        else:
            parts = text[x:]

        #Here we have broken down our text in to 2000 character chunks

        # Azure portal URL.
        base_url = 'https://westus.api.cognitive.microsoft.com/'
        # The unique account key goes here.
        account_key = '01f5b19c9bd9425ab5c17dfaf2a497ed'

        headers = {'Content-Type':'application/json', 'Ocp-Apim-Subscription-Key':account_key}

        input_texts = '{"documents":[{"id":"1","text":"'+parts+'"}]}' 
        #Here, instead of manually inputting any codes, we have put in the 2000 character chunk

        # Detect key phrases.
        batch_keyphrase_url = base_url + 'text/analytics/v2.0/keyPhrases'
        req = urllib2.Request(batch_keyphrase_url, input_texts, headers) 
        response = urllib2.urlopen(req)
        result = response.read()
        keyph = json.loads(result)

        for phrases in keyph['documents']:
            keyphrases3.append(phrases['keyPhrases'])

        # Detect sentiment.
        batch_sentiment_url = base_url + 'text/analytics/v2.0/sentiment'
        req = urllib2.Request(batch_sentiment_url, input_texts, headers) 
        response = urllib2.urlopen(req)
        result = response.read()
        senti = json.loads(result)

        for sentiment_analysis in senti['documents']:
            sentiment3.append(sentiment_analysis['score']) # The sentiment scores are appended to the sentiment list

    # The keyphrases list will be a list within a list, containing a set of keywords obtained after every single run of 2000
    # characters. So now, we will iterate over these inner lists to get each of the keywords into a final list

    for items in keyphrases2:
        for x in items:
            final2.append(x)

    unique_keys = list(set(final3)) #Using set functions, we clean the list to remove duplicates

    total_keys3.append(unique_keys) #In this list we will be appending the list of unique keywords for each of the 10K submissions

    avg_senti = np.mean(sentiment3) #Here we are calculate the average sentiment score for each of the chunks

    avg_score3.append(avg_senti) # Storing the average sentiment score in the average score list

In [37]:
avg_score3

[0.67097830772399902]

In [2]:
# Azure portal URL.
base_url = 'https://westus.api.cognitive.microsoft.com/'
# Your account key goes here.
account_key = '01f5b19c9bd9425ab5c17dfaf2a497ed'

headers = {'Content-Type':'application/json', 'Ocp-Apim-Subscription-Key':account_key}
            
input_texts = '{"documents":[{"id":"1","text":"This was one of the best restaurants I have been to.  the food was amazing!"},\
                {"id":"2","text":"This restaurant was ok.  Nothing to write home about."},\
                {"id":"three","text":"This was the worst restaurant I have been to.  Truly horrible."}]}'